In [91]:

import json

# from keras.models import Model
from keras.layers import Dense, Input, concatenate, Flatten, Multiply, Dropout
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
# import keras.callbacks as cb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from joblib import dump, load
import critic_model

import numpy as np

In [53]:
def load_data(name):
    with open(name) as json_file:
         data = json.load(json_file)
    
    
    data = data['data']
        
        
        
    new_all_game_situation = np.zeros((len(data), len(data[0]['array_input'][0])))
    for i in range(len(data)):
        new_all_game_situation[i] = data[i]['array_input'][0]
                

        
        
    new_card_to_use = np.zeros((len(data), len(data[0]['gen_my_all_card'][0])))
    for i in range(len(data)):
        new_card_to_use[i] = data[i]['gen_my_all_card'][0]
        
        
        
        
        
    new_all_rewards = np.zeros((len(data), 1), dtype=np.dtype('int64'))
    for i in range(len(data)):
        new_all_rewards[i] = data[i]['reward']
    
    #new_action_val_array = np.zeros((len(data), len(data[0]['action_val'])))
    #for i in range(len(data)):
    #    ind = np.argmax(data[i]['action_val'])
    #    new_action_val_array[i][ind] = 1
    
    gen_all_card_second = np.zeros((len(data), len(data[0]['gen_all_card_second'][0])))
    for i in range(len(data)):
        gen_all_card_second[i] = data[i]['gen_all_card_second'][0]
    

    card_deck = np.zeros((len(data), len(data[0]['card_deck'][0])))
    for i in range(len(data)):
        card_deck[i] = data[i]['card_deck'][0]
        
    new_all_game_situation = np.concatenate([new_all_game_situation, new_card_to_use - gen_all_card_second], axis=1)
    return new_all_game_situation, new_all_rewards

In [15]:
import json
import keras

In [16]:
X_test, y_test = load_data('data_2000.txt')
X_train, y_train = load_data('data_4000.txt')

In [17]:
X_train, _, y_train, _ = train_test_split(X_test, y_test, test_size=0.9)

In [203]:
X_test, y_test = load_data('data_2000.txt')

In [54]:
X_train_big_1, y_train_big_1 = load_data('data_big_4000.txt')

In [55]:
X_train_big_2, y_train_big_2 = load_data('data_big_8000.txt')

In [56]:
X_train_big_3, y_train_big_3 = load_data('data_big_12000.txt')

In [57]:
X_train_big_4, y_train_big_4 = load_data('data_big_16000.txt')

In [58]:
X_test, y_test = load_data('data_big_20000.txt')

In [59]:
X_train = np.concatenate([X_train_big_1, X_train_big_2, X_train_big_3, X_train_big_4], axis=0)

In [60]:
y_train = np.concatenate([y_train_big_1, y_train_big_2, y_train_big_3, y_train_big_4], axis=0)

In [75]:
X_train = np.concatenate([X_train_big_1], axis=0)
y_train = np.concatenate([y_train_big_1], axis=0)

In [37]:
#X_train, y_train = X_train_big_1, y_train_big_1

In [38]:
#X_test, y_test = X_train_big_2, y_train_big_2

In [85]:
from keras.layers import Input, Dense, Flatten, Reshape, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD

input = Input(shape=(len(X_train[0]),))
bn = BatchNormalization()(input)
lr1 = Dense(1024, activation='relu')(bn)
do1 = Dropout(0.5)(lr1)
#lr2 = Dense(5000, activation='relu')(do1)
#do2 = Dropout(0.5)(lr1)
#lr3 = Dense(5000, activation='relu')(do2)
#do3 = Dropout(0.5)(lr1)
#lr4 = Dense(5000, activation='relu')(do3)
#do4 = Dropout(0.5)(lr1)
#lr5 = Dense(5000, activation='relu')(do4)
#do5 = Dropout(0.5)(lr1)
#lr6 = Dense(5000, activation='relu')(do5)
#do6 = Dropout(0.5)(lr1)
output = Dense(1, activation='tan')(do1)

model = Model(input, output, name="encoder")


opt = SGD(lr=0.01)
model.compile(optimizer=opt, loss='mse', metrics=['mse'])

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 5)]
model.fit(X_train, y_train,
                epochs=10000,
                shuffle=True,
                batch_size=2048,
                validation_data=(X_test, y_test), callbacks=callbacks)

Train on 84497 samples, validate on 21176 samples
Epoch 1/10000
84497/84497 [==============================] - 6s 77us/step - loss: 1.3046 - mse: 1.3046 - val_loss: 0.8626 - val_mse: 0.8626
Epoch 2/10000
84497/84497 [==============================] - 6s 67us/step - loss: 1.0596 - mse: 1.0596 - val_loss: 0.8136 - val_mse: 0.8136
Epoch 3/10000
84497/84497 [==============================] - 6s 73us/step - loss: 0.9671 - mse: 0.9671 - val_loss: 0.7980 - val_mse: 0.7980
Epoch 4/10000
84497/84497 [==============================] - 6s 69us/step - loss: 0.9153 - mse: 0.9153 - val_loss: 0.7905 - val_mse: 0.7905
Epoch 5/10000
84497/84497 [==============================] - 6s 68us/step - loss: 0.8833 - mse: 0.8833 - val_loss: 0.7865 - val_mse: 0.7865
Epoch 6/10000
84497/84497 [==============================] - 6s 75us/step - loss: 0.8598 - mse: 0.8598 - val_loss: 0.7828 - val_mse: 0.7828
Epoch 7/10000
84497/84497 [==============================] - 6s 75us/step - loss: 0.8427 - mse: 0.8427 - val_l

In [72]:
model.fit(X_train, y_train,
                epochs=10000,
                shuffle=True,
                batch_size=2048,
                validation_data=(X_test, y_test), callbacks=callbacks)

Train on 84497 samples, validate on 21176 samples
Epoch 1/10000
84497/84497 [==============================] - 5s 59us/step - loss: 0.6798 - mse: 0.8427 - val_loss: 0.6855 - val_mse: 0.8330
Epoch 2/10000
84497/84497 [==============================] - 5s 61us/step - loss: 0.6791 - mse: 0.8436 - val_loss: 0.6840 - val_mse: 0.8328
Epoch 3/10000
84497/84497 [==============================] - 5s 64us/step - loss: 0.6780 - mse: 0.8438 - val_loss: 0.6825 - val_mse: 0.8326
Epoch 4/10000
84497/84497 [==============================] - 5s 64us/step - loss: 0.6767 - mse: 0.8434 - val_loss: 0.6811 - val_mse: 0.8326
Epoch 5/10000
84497/84497 [==============================] - 5s 63us/step - loss: 0.6733 - mse: 0.8390 - val_loss: 0.6798 - val_mse: 0.8325
Epoch 6/10000
84497/84497 [==============================] - 5s 64us/step - loss: 0.6747 - mse: 0.8438 - val_loss: 0.6785 - val_mse: 0.8325
Epoch 7/10000
84497/84497 [==============================] - 5s 61us/step - loss: 0.6710 - mse: 0.8394 - val_l

KeyboardInterrupt: 

In [21]:
print(mean_squared_error(y_test, model.predict(X_test)))

0.5274446227472405


In [73]:
model.predict(X_test)[:15]

array([[ 0.29196808],
       [-0.10801058],
       [ 0.5519429 ],
       [ 0.00113484],
       [-0.38529956],
       [-0.3805221 ],
       [ 0.14772765],
       [-0.95421416],
       [-0.71315604],
       [ 0.67042935],
       [-0.7349194 ],
       [ 0.57917446],
       [-0.9725838 ],
       [-0.48480994],
       [ 0.08816356]], dtype=float32)

In [53]:
y_test[:15]

array([[ 1],
       [ 1],
       [-1],
       [-1],
       [ 1],
       [-1],
       [-1],
       [ 1],
       [ 1],
       [-1],
       [-1],
       [ 1],
       [-1],
       [ 1],
       [-1]], dtype=int64)

In [70]:
critic = load('critic_loss_0.660.joblib') 

In [71]:
print(mean_squared_error(y_test, critic.predict(X_test)))

0.6973338664458024


In [105]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor

In [106]:
est = HistGradientBoostingRegressor(verbose=2, max_iter=200)
est.fit(X_train, y_train.ravel())
print(mean_squared_error(y_test, est.predict(X_test)))

Binning 0.038 GB of data: 0.465 s
Fitting gradient boosted rounds:
[1/200] 1 tree, 31 leaves, max depth = 9, in 0.119s
[2/200] 1 tree, 31 leaves, max depth = 9, in 0.120s
[3/200] 1 tree, 31 leaves, max depth = 8, in 0.106s
[4/200] 1 tree, 31 leaves, max depth = 8, in 0.109s
[5/200] 1 tree, 31 leaves, max depth = 8, in 0.106s
[6/200] 1 tree, 31 leaves, max depth = 10, in 0.097s
[7/200] 1 tree, 31 leaves, max depth = 9, in 0.125s
[8/200] 1 tree, 31 leaves, max depth = 7, in 0.102s
[9/200] 1 tree, 31 leaves, max depth = 9, in 0.092s
[10/200] 1 tree, 31 leaves, max depth = 8, in 0.108s
[11/200] 1 tree, 31 leaves, max depth = 8, in 0.125s
[12/200] 1 tree, 31 leaves, max depth = 10, in 0.105s
[13/200] 1 tree, 31 leaves, max depth = 9, in 0.111s
[14/200] 1 tree, 31 leaves, max depth = 13, in 0.111s
[15/200] 1 tree, 31 leaves, max depth = 11, in 0.119s
[16/200] 1 tree, 31 leaves, max depth = 12, in 0.116s
[17/200] 1 tree, 31 leaves, max depth = 9, in 0.097s
[18/200] 1 tree, 31 leaves, max dept

[151/200] 1 tree, 31 leaves, max depth = 11, in 0.093s
[152/200] 1 tree, 31 leaves, max depth = 9, in 0.090s
[153/200] 1 tree, 31 leaves, max depth = 15, in 0.093s
[154/200] 1 tree, 31 leaves, max depth = 17, in 0.077s
[155/200] 1 tree, 31 leaves, max depth = 14, in 0.115s
[156/200] 1 tree, 31 leaves, max depth = 14, in 0.084s
[157/200] 1 tree, 31 leaves, max depth = 13, in 0.088s
[158/200] 1 tree, 31 leaves, max depth = 12, in 0.080s
[159/200] 1 tree, 31 leaves, max depth = 8, in 0.092s
[160/200] 1 tree, 31 leaves, max depth = 17, in 0.085s
[161/200] 1 tree, 31 leaves, max depth = 13, in 0.084s
[162/200] 1 tree, 31 leaves, max depth = 11, in 0.086s
[163/200] 1 tree, 31 leaves, max depth = 12, in 0.076s
[164/200] 1 tree, 31 leaves, max depth = 14, in 0.092s
[165/200] 1 tree, 31 leaves, max depth = 15, in 0.079s
[166/200] 1 tree, 31 leaves, max depth = 18, in 0.097s
[167/200] 1 tree, 31 leaves, max depth = 16, in 0.082s
[168/200] 1 tree, 31 leaves, max depth = 12, in 0.199s
[169/200] 1 

array([[ 0.10712602],
       [-0.12243053],
       [ 0.3452082 ],
       [-0.25971276],
       [-0.28437054],
       [ 0.3338394 ],
       [-0.20273483],
       [ 0.43285337],
       [-0.5641596 ],
       [ 0.43943235],
       [ 0.4847895 ],
       [-0.49727315],
       [-0.55646074],
       [ 0.5392559 ],
       [-0.509521  ],
       [ 0.50222296],
       [-0.37170887],
       [ 0.4034697 ],
       [-0.43109402],
       [ 0.07978395],
       [-0.0818106 ],
       [ 0.38821605],
       [-0.42126086],
       [ 0.59114385],
       [-0.65338844],
       [ 0.6398108 ],
       [-0.6719248 ],
       [ 0.5704599 ],
       [-0.5946426 ],
       [-0.66352934],
       [ 0.46705028],
       [-0.42759892],
       [ 0.07293329],
       [-0.10713087],
       [ 0.28907755],
       [-0.52731377],
       [ 0.615381  ],
       [-0.6931366 ],
       [ 0.49481955],
       [-0.28086057],
       [ 0.4554372 ],
       [-0.49119422],
       [ 0.4830594 ],
       [-0.5210782 ],
       [ 0.4650689 ],
       [-0

In [208]:
model.predict(X_test[:10])

array([[-0.5789287 ],
       [-0.32958326],
       [ 0.64002913],
       [-0.8431802 ],
       [ 0.80426174],
       [-0.79008085],
       [ 0.91677564],
       [-0.8998283 ],
       [ 0.98294514],
       [-0.9849505 ]], dtype=float32)

In [79]:
from keras.layers import Input, Dense, Flatten, Reshape, Dropout, BatchNormalization
from keras.models import Model
input = Input(shape=(220,))
bn = BatchNormalization()(input)
lr1 = Dense(256, activation='relu')(bn)
output = Dense(1, activation='tanh')(lr1)

model = Model(input, output)
model.compile(optimizer='adam', loss='mse')

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 2)]
model.fit(X_train, y_train,
                epochs=100,
                shuffle=True,
                batch_size=2048,
                validation_data=(X_test, y_test), callbacks=callbacks)

Train on 21361 samples, validate on 21176 samples
Epoch 1/100
21361/21361 [==============================] - 1s 52us/step - loss: 1.4354 - val_loss: 0.9443
Epoch 2/100
21361/21361 [==============================] - 1s 29us/step - loss: 1.2457 - val_loss: 0.8963
Epoch 3/100
21361/21361 [==============================] - 1s 30us/step - loss: 1.1442 - val_loss: 0.8914
Epoch 4/100
21361/21361 [==============================] - 1s 31us/step - loss: 1.0637 - val_loss: 0.8629
Epoch 5/100
21361/21361 [==============================] - 1s 29us/step - loss: 0.9996 - val_loss: 0.8390
Epoch 6/100
21361/21361 [==============================] - 1s 30us/step - loss: 0.9504 - val_loss: 0.8305
Epoch 7/100
21361/21361 [==============================] - 1s 29us/step - loss: 0.9178 - val_loss: 0.8241
Epoch 8/100
21361/21361 [==============================] - 1s 27us/step - loss: 0.8839 - val_loss: 0.8099
Epoch 9/100
21361/21361 [==============================] - 1s 27us/step - loss: 0.8566 - val_loss: 0.8

In [80]:
from keras.layers import Input, Dense, Flatten, Reshape, Dropout, BatchNormalization
from keras.models import Model
input = Input(shape=(len(X_train[0]),))
bn = BatchNormalization()(input)
lr1 = Dense(512, activation='tanh')(bn)
do1 = Dropout(0.75)(lr1)
output = Dense(1, activation='tanh')(do1)

model = Model(input, output, name="encoder")
model.compile(optimizer='adam', loss='mse')

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 2)]
model.fit(X_train, y_train,
                epochs=100,
                shuffle=True,
                batch_size=2048,
                validation_data=(X_test, y_test), callbacks=callbacks)

Train on 21361 samples, validate on 21176 samples
Epoch 1/100
21361/21361 [==============================] - 1s 61us/step - loss: 1.3946 - val_loss: 0.9580
Epoch 2/100
21361/21361 [==============================] - 1s 45us/step - loss: 1.1548 - val_loss: 0.9123
Epoch 3/100
21361/21361 [==============================] - 1s 43us/step - loss: 1.0605 - val_loss: 0.9035
Epoch 4/100
21361/21361 [==============================] - 1s 43us/step - loss: 0.9865 - val_loss: 0.9062
Epoch 5/100
21361/21361 [==============================] - 1s 43us/step - loss: 0.9460 - val_loss: 0.9068


In [103]:
from keras.layers import Input, Dense, Flatten, Reshape, Dropout, BatchNormalization
from keras.models import Model
input = Input(shape=(len(X_test[0]),))
bn = BatchNormalization()(input)
lr1 = Dense(256, activation='relu')(bn)
do1 = Dropout(0.5)(lr1)
output = Dense(1, activation='tanh')(do1)

model = Model(input, output, name="encoder")
model.compile(optimizer='adam', loss='mse')

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 2)]
model.fit(X_train, y_train,
                epochs=100,
                shuffle=True,
                batch_size=2048,
                validation_data=(X_test, y_test), callbacks=callbacks)

Train on 84497 samples, validate on 21176 samples
Epoch 1/100
84497/84497 [==============================] - 3s 36us/step - loss: 1.5549 - val_loss: 0.8481
Epoch 2/100
84497/84497 [==============================] - 2s 30us/step - loss: 1.3248 - val_loss: 0.8244
Epoch 3/100
84497/84497 [==============================] - 3s 30us/step - loss: 1.1554 - val_loss: 0.7992
Epoch 4/100
84497/84497 [==============================] - 2s 26us/step - loss: 1.0136 - val_loss: 0.7617
Epoch 5/100
84497/84497 [==============================] - 2s 27us/step - loss: 0.8878 - val_loss: 0.7632
Epoch 6/100
84497/84497 [==============================] - 2s 26us/step - loss: 0.8248 - val_loss: 0.7669


In [85]:
from keras.layers import Input, Dense, Flatten, Reshape, Dropout, BatchNormalization
from keras.models import Model
input = Input(shape=(len(X_train[0]),))
bn = BatchNormalization()(input)
lr1 = Dense(1024, activation='relu')(bn)
do1 = Dropout(0.75)(lr1)
output = Dense(1, activation='tanh')(do1)

model = Model(input, output, name="encoder")
model.compile(optimizer='adam', loss='mse')

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 2)]
model.fit(X_train, y_train,
                epochs=100,
                shuffle=True,
                batch_size=2048,
                validation_data=(X_test, y_test), callbacks=callbacks)

Train on 21361 samples, validate on 21176 samples
Epoch 1/100
21361/21361 [==============================] - 2s 98us/step - loss: 1.2278 - val_loss: 0.8770
Epoch 2/100
21361/21361 [==============================] - 2s 71us/step - loss: 1.0067 - val_loss: 0.8974
Epoch 3/100
21361/21361 [==============================] - 2s 72us/step - loss: 0.9129 - val_loss: 0.8832


In [86]:
from keras.layers import Input, Dense, Flatten, Reshape, Dropout, BatchNormalization
from keras.models import Model
input = Input(shape=(len(X_train[0]),))
bn = BatchNormalization()(input)
lr1 = Dense(1024, activation='relu')(bn)
do1 = Dropout(0.75)(lr1)
lr2 = Dense(1024, activation='relu')(do1)
do2 = Dropout(0.75)(lr2)
output = Dense(1, activation='tanh')(do2)

model = Model(input, output, name="encoder")
model.compile(optimizer='adam', loss='mse')

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 2)]
model.fit(X_train, y_train,
                epochs=100,
                shuffle=True,
                batch_size=4096,
                validation_data=(X_test, y_test), callbacks=callbacks)

Train on 21361 samples, validate on 21176 samples
Epoch 1/100
21361/21361 [==============================] - 5s 230us/step - loss: 1.4575 - val_loss: 1.0030
Epoch 2/100
21361/21361 [==============================] - 4s 191us/step - loss: 1.2463 - val_loss: 0.9524
Epoch 3/100
21361/21361 [==============================] - 5s 227us/step - loss: 1.1635 - val_loss: 0.8929
Epoch 4/100
21361/21361 [==============================] - 5s 220us/step - loss: 1.0804 - val_loss: 0.8590
Epoch 5/100
21361/21361 [==============================] - 5s 217us/step - loss: 1.0148 - val_loss: 0.8416
Epoch 6/100
21361/21361 [==============================] - 5s 223us/step - loss: 0.9649 - val_loss: 0.8010
Epoch 7/100
21361/21361 [==============================] - 5s 226us/step - loss: 0.9249 - val_loss: 0.7914
Epoch 8/100
21361/21361 [==============================] - 5s 214us/step - loss: 0.8740 - val_loss: 0.7867
Epoch 9/100
21361/21361 [==============================] - 5s 215us/step - loss: 0.8434 - val_

In [92]:
board_input = Input(shape=(2 + 2 + 6 + 6,))
action_input = Input(shape=(34 * 3 * 2,), name='action_input')
    
bn = concatenate([BatchNormalization()(board_input), action_input], axis=1)
lr1 = Dense(256, activation='relu')(bn)
value_output = Dense(1, activation='tanh')(lr1)

model = Model(inputs=[board_input, action_input], outputs=value_output)
    
opt = Adam(lr=0.00001)
model.compile(loss='mse', optimizer=opt)

In [94]:
X_test_1 = X_test[:,:16]

In [95]:
X_test_1

array([[37., 18., 20., ..., 11.,  6., 14.],
       [37., 10., 21., ..., 10.,  2.,  7.],
       [33.,  6., 48., ..., 10.,  3., 10.],
       ...,
       [28., 24., 43., ..., 53.,  4.,  1.],
       [28., 17., 43., ..., 45.,  1.,  4.],
       [38.,  6., 40., ..., 13., 11., 12.]])

In [96]:
X_test_2 = X_test[:,16:]

In [98]:
model.load_weights('critic.h5')

In [100]:
print(mean_squared_error(y_test, model.predict([X_test_1, X_test_2])))

0.8365375673230624


In [101]:
0.8365375673230624**0.5

0.9146242765874205

In [9]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split

%matplotlib inline
#random seed as the birthday of my granp which is in the hospital fighting with cancer
#be strong Valdomiro!
np.random.seed(10171927)
tf.random.set_seed(10171927)

#to see how long the notebook lasts to run
start = time.time()

In [ ]:
print('TensorFlow version (expected = 2.0.0):', tf.__version__)
print('TensorFlow Probability version (expected = 0.9.0-dev20190912):', tfp.__version__)

In [35]:
cp_x, cp_y = X_train, y_train

In [ ]:
X_train, y_train = cp_x[:10000], cp_y[:10000]

In [87]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split

In [88]:
def build_bayesian_bcnn_model(input_shape):
    
    """
    Here we use tf.keras.Model to use our graph as a Neural Network:
    We select our input node as the net input, and the last node as our output (predict node).
    Note that our model won't be compiled, as we are usign TF2.0 and will optimize it with
    a custom @tf.function for loss and a @tf.function for train_step
    Our input parameter is just the input shape, a tuple, for the input layer
    """
    
    model_in = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.BatchNormalization()(model_in)
    x = tf.keras.layers.Activation('relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Activation('relu')(x)
    dense_1 = tfp.python.layers.DenseFlipout(128, activation='relu')
    x = dense_1(x)
    dense_2 = tfp.python.layers.DenseFlipout(1, activation='tanh')
    model_out = dense_2(x)  # logits
    model = tf.keras.Model(model_in, model_out)
    return model

In [89]:
@tf.function
def elbo_loss(labels, logits):
    #loss_en = tf.nn.softmax_cross_entropy_with_logits(labels, logits)
    #loss_kl = tf.keras.losses.KLD(labels, logits)
    loss_kl = tf.keras.losses.MSE(labels, logits)
    #loss = tf.reduce_mean(tf.add(loss_en, loss_kl))
    loss = tf.reduce_mean(loss_kl)
    return loss

In [90]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = bcnn(X_train)
        loss = elbo_loss(labels, logits)
    gradients = tape.gradient(loss, bcnn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, bcnn.trainable_variables))
    return loss

#def accuracy(preds, labels):
#    return np.mean(np.argmax(preds, axis=1) == np.argmax(labels, axis=1))

In [91]:
bcnn = build_bayesian_bcnn_model(X_train.shape[1:])
optimizer = tf.keras.optimizers.Adam(lr=0.01)

Instructions for updating:
Please use `layer.add_weight` method instead.


In [92]:
#y_train_big_1 = keras.utils.to_categorical(y_train_big_1, 2)
#y_train_big_2 = keras.utils.to_categorical(y_train_big_2, 2)

In [93]:

times = []
accs = []
val_accs = []
losses = []
val_losses = []
for i in range(1000):
    tic = time.time()
    loss = train_step(X_train, y_train)
    losses.append(loss)
    
    val_preds = bcnn(X_test)
    val_loss = elbo_loss(y_test, val_preds)
    #val_acc = accuracy(y_train_big_1, val_preds)
    print(val_preds[:10], y_test[:10])
    
    #val_accs.append(val_acc)
    val_losses.append(val_loss)
    tac = time.time()
    train_time = tac-tic
    times.append(train_time)
    
    print("Epoch: {}: loss = {:7.3f}, val_loss = {:7.3f}, time: {:7.3f}".format(i, loss, val_loss, train_time))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(
[[0.99983597]
 [0.9999991 ]
 [0.99907553]
 [0.31383407]
 [0.99991256]
 [0.9996051 ]
 [0.9162453 ]
 [0.981101  ]
 [0.9961533 ]
 [0.9969849 ]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 0: loss =   1.853, val_loss =   1.834, time:   6.169
tf.Tensor(
[[ 0.99842376]
 [ 0.9742869 ]
 [-0.99999636]
 [-0.9988516 ]
 [ 0.18709797]
 [ 0.99970883]
 [ 0.9995341 ]
 [-0.9901253 ]
 [ 0.43300477]
 [-0.99998057]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]

tf.Tensor(
[[ 0.9999989 ]
 [ 0.9934065 ]
 [-0.54466623]
 [ 0.4654407 ]
 [ 0.2010665 ]
 [-0.9363418 ]
 [ 0.9181771 ]
 [-0.9969605 ]
 [-0.97059804]
 [ 0.99999255]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 24: loss =   1.103, val_loss =   1.159, time:   1.157
tf.Tensor(
[[ 1.        ]
 [ 0.96054405]
 [ 0.777756  ]
 [-0.98774415]
 [-0.9829916 ]
 [-0.9873745 ]
 [ 0.9997976 ]
 [-0.9737393 ]
 [-0.99983025]
 [ 0.9987641 ]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 25: loss =   1.126, val_loss =   1.134, time:   1.035
tf.Tensor(
[[ 0.9998997 ]
 [ 0.9970043 ]
 [-0.6706757 ]
 [-0.97228193]
 [-0.8586792 ]
 [-0.9968167 ]
 [ 0.97371364]
 [-0.9830823 ]
 [-0.9747187 ]
 [ 0.99992305]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 26: loss =   1.113, val_loss =   1.131, time:   1.060
tf.Tensor(
[[ 0.99991935]
 [ 0.9970339 ]
 [-0.67523956]

tf.Tensor(
[[ 0.73727643]
 [ 0.7450168 ]
 [-0.30071446]
 [-0.02155814]
 [-0.08213896]
 [-0.5444006 ]
 [ 0.54316854]
 [-0.85326576]
 [-0.47728252]
 [ 0.4881586 ]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 51: loss =   0.813, val_loss =   0.862, time:   1.256
tf.Tensor(
[[ 0.8099113 ]
 [ 0.4924748 ]
 [-0.59090716]
 [-0.5130276 ]
 [-0.5840848 ]
 [-0.606215  ]
 [ 0.67696446]
 [-0.92513025]
 [-0.96689874]
 [ 0.7486763 ]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 52: loss =   0.807, val_loss =   0.879, time:   1.235
tf.Tensor(
[[ 0.30446967]
 [ 0.5017291 ]
 [-0.6633931 ]
 [-0.47156236]
 [-0.0795053 ]
 [-0.859576  ]
 [ 0.34515885]
 [-0.9346589 ]
 [-0.9595519 ]
 [ 0.7000551 ]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 53: loss =   0.819, val_loss =   0.861, time:   1.249
tf.Tensor(
[[ 0.83338004]
 [ 0.75809765]
 [-0.5266529 ]

tf.Tensor(
[[ 0.85246736]
 [ 0.6061873 ]
 [ 0.02987074]
 [-0.39426684]
 [-0.31889716]
 [-0.56150985]
 [ 0.5348409 ]
 [-0.9641187 ]
 [ 0.08328824]
 [ 0.9853317 ]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 78: loss =   0.761, val_loss =   0.823, time:   1.303
tf.Tensor(
[[ 0.85331994]
 [ 0.6623335 ]
 [ 0.15209658]
 [ 0.38062465]
 [-0.08191635]
 [ 0.31146675]
 [ 0.6399118 ]
 [-0.8268905 ]
 [ 0.24549723]
 [ 0.99382347]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 79: loss =   0.747, val_loss =   0.828, time:   1.131
tf.Tensor(
[[ 0.87770003]
 [ 0.6341458 ]
 [-0.0361998 ]
 [ 0.3349615 ]
 [ 0.01662813]
 [-0.82522166]
 [ 0.43156976]
 [-0.94003177]
 [ 0.15081604]
 [ 0.88516426]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 80: loss =   0.736, val_loss =   0.808, time:   1.466
tf.Tensor(
[[ 0.71409434]
 [ 0.11440261]
 [-0.04123497]

tf.Tensor(
[[ 0.8044319 ]
 [ 0.3897777 ]
 [ 0.0863558 ]
 [ 0.04126542]
 [-0.21814384]
 [-0.3418143 ]
 [ 0.2940084 ]
 [-0.9765976 ]
 [-0.64335465]
 [ 0.57396775]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 104: loss =   0.694, val_loss =   0.815, time:   1.283
tf.Tensor(
[[ 0.9495794 ]
 [ 0.64082265]
 [ 0.17601685]
 [ 0.377904  ]
 [-0.222141  ]
 [-0.22968316]
 [ 0.4598488 ]
 [-0.98509747]
 [-0.5814592 ]
 [ 0.6938946 ]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 105: loss =   0.699, val_loss =   0.814, time:   1.258
tf.Tensor(
[[ 0.9214706 ]
 [ 0.5312642 ]
 [ 0.25378704]
 [ 0.17668104]
 [-0.04760589]
 [-0.373328  ]
 [ 0.62283826]
 [-0.9765748 ]
 [ 0.31896988]
 [ 0.80637234]], shape=(10, 1), dtype=float32) [[ 1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [-1]]
Epoch: 106: loss =   0.692, val_loss =   0.820, time:   1.274
tf.Tensor(
[[ 0.96167237]
 [ 0.6833956 ]
 [-0.068274

KeyboardInterrupt: 

In [ ]:
# Plot training & validation loss values
plt.plot(np.array(losses), label="loss")
plt.plot(np.array(val_losses), label="val_loss")
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [18]:
#bcnn.save_weights("critic.h5")

In [59]:
arr = bcnn(X_test)

In [15]:
print(mean_squared_error(y_test, bcnn(X_test)))

0.4405348770485342


In [16]:
bcnn(X_test)

<tf.Tensor: shape=(32044, 1), dtype=float32, numpy=
array([[-0.9997102],
       [-0.9884574],
       [-0.8457604],
       ...,
       [ 1.       ],
       [-1.       ],
       [ 1.       ]], dtype=float32)>

In [20]:
X_test[0].shape

(424,)

In [22]:
keras.utils.to_categorical(y_train_big_1.reshape(len(y_train_big_1)) + np.ones((len(y_train_big_1),)), 2)

IndexError: index 2 is out of bounds for axis 1 with size 2

In [8]:
import keras

In [27]:
arr = y_train_big_1.reshape(len(y_train_big_1)) + np.ones((len(y_train_big_1)))

In [29]:
for i in range(len(arr)):
    if arr[i] == 2:
        arr[i] = 1

In [31]:
y_train_big = keras.utils.to_categorical(arr, 2)

In [33]:
arr = y_train_big_2.reshape(len(y_train_big_2)) + np.ones((len(y_train_big_2)))
for i in range(len(arr)):
    if arr[i] == 2:
        arr[i] = 1
y_test = keras.utils.to_categorical(arr, 2)

In [94]:
import tensorflow as tf

In [95]:
kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (x.shape[0] * 1.0)
bias_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (x.shape[0] * 1.0)

model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],), name="basket"),
    tf.keras.layers.BatchNormalization(),
    tfp.layers.DenseFlipout(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tfp.layers.DenseFlipout(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tfp.layers.DenseFlipout(1, activation='tanh', kernel_divergence_fn=kl_divergence_function),
    #tfp.layers.DenseFlipout(1, activation='tanh')
])

NameError: name 'kl_divergence_function' is not defined

In [39]:
learning_rate = 1.0e-3

model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['mse'])

In [40]:
model.fit(X_train, y_train, epochs=100, batch_size=1024, validation_data=(X_test, y_test))

Train on 214406 samples, validate on 212198 samples
Epoch 1/100
214406/214406 [==============================] - 61s 285us/sample - loss: 422344.7958 - mse: 1.4628 - val_loss: 404302.2318 - val_mse: 1.0751
Epoch 2/100
214406/214406 [==============================] - 53s 246us/sample - loss: 386860.3004 - mse: 1.2647 - val_loss: 369235.6837 - val_mse: 1.1515
Epoch 3/100
 56320/214406 [======>.......................] - ETA: 21s - loss: 364848.4601 - mse: 1.2768

KeyboardInterrupt: 

In [44]:
mean_squared_error(y_test, model.predict(X_test))

1.1782597842483173

array([1], dtype=int64)

In [96]:
from tensorflow.keras.layers import Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [97]:
tfd = tfp.distributions
tfb = tfp.bijectors

def NLL(y, distr): 
    return -distr.log_prob(y) 

def normal_sp(params): 
    return tfd.Normal(loc=params[:,0:1], scale=1e-3 + tf.math.softplus(0.05 * params[:,1:2]))# both parameters are learnable

kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X_train.shape[0] * 1.0)
bias_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X_train.shape[0] * 1.0)


inputs = Input(shape=(X_train.shape[1],))

inp = BatchNormalization()(inputs)
hidden = tfp.layers.DenseFlipout(256,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(inp)
do1 = tf.keras.layers.Dropout(0.2)(hidden)
params = tfp.layers.DenseFlipout(1,activation="tanh", bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn)(do1)
#dist = tfp.layers.DistributionLambda(normal_sp)(params) 


model_vi = Model(inputs=inputs, outputs=params)
#model_vi.compile(Adam(learning_rate=0.001), loss=NLL, metrics=['mse']) 
model_vi.compile(Adam(learning_rate=0.01), loss='mse', metrics=['mse']) 
model_params = Model(inputs=inputs, outputs=params)

In [98]:
model_vi.fit(X_train, y_train, epochs=100, batch_size=1024, validation_data=(X_test, y_test))

Train on 21361 samples, validate on 21176 samples
Epoch 1/100
21361/21361 [==============================] - 6s 276us/sample - loss: 7.8212 - mse: 1.2206 - val_loss: 7.5987 - val_mse: 1.1430
Epoch 2/100
21361/21361 [==============================] - 2s 105us/sample - loss: 7.3253 - mse: 1.0002 - val_loss: 7.3217 - val_mse: 1.1426
Epoch 3/100
21361/21361 [==============================] - 2s 100us/sample - loss: 6.9912 - mse: 0.9433 - val_loss: 6.9912 - val_mse: 1.0894
Epoch 4/100
21361/21361 [==============================] - 2s 102us/sample - loss: 6.6623 - mse: 0.8903 - val_loss: 6.6332 - val_mse: 1.0049
Epoch 5/100
21361/21361 [==============================] - 2s 100us/sample - loss: 6.3438 - mse: 0.8428 - val_loss: 6.2884 - val_mse: 0.9283
Epoch 6/100
21361/21361 [==============================] - 2s 99us/sample - loss: 6.0122 - mse: 0.7763 - val_loss: 5.9627 - val_mse: 0.8644
Epoch 7/100
21361/21361 [==============================] - 2s 109us/sample - loss: 5.6973 - mse: 0.7211 -

KeyboardInterrupt: 

In [ ]:
mean_squared_error(y_test, model_vi.predict(X_test))

In [120]:
model_vi.predict(X_test[:10].reshape(10, -1))

array([[ 0.110635  ],
       [ 0.31213433],
       [ 0.4475886 ],
       [-0.34117684],
       [ 0.02357351],
       [-0.1377169 ],
       [-0.00382401],
       [ 0.35468742],
       [-0.23620737],
       [-0.08607194]], dtype=float32)

In [95]:
y_test[:10]

array([[ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.]])

In [72]:
y_train = y_train.astype(float)

In [78]:
y_test = y_test.astype(float)

In [81]:
mean_squared_error(y_test, model_vi.predict(X_test))

1.542389368827469

In [82]:
model_vi.predict(X_test[:10].reshape(10, -1))

array([[-0.32806292],
       [-0.46644053],
       [-1.4164567 ],
       [ 0.09548746],
       [-0.5884465 ],
       [ 0.29557213],
       [-0.01731414],
       [ 0.41492966],
       [ 0.88287306],
       [ 0.21522072]], dtype=float32)

In [99]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
  

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(424,)),
    tfp.layers.DenseFlipout(256, activation='relu'),
    tfp.layers.DenseFlipout(units=1, activation='tanh')
])

In [100]:

model.compile(loss='mse', optimizer=Adam(lr=0.03), metrics=['mse'])


In [101]:
model.fit(X_train, y_train, epochs=100, batch_size=1024, validation_data=(X_test, y_test))

ValueError: Error when checking input: expected input_37 to have shape (424,) but got array with shape (220,)

In [172]:
model.predict(X_test[:10].reshape(10, -1))

array([[-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.]], dtype=float32)